In [4]:
server_name = sql_db_server_name

### Initializations

In [5]:
import pyodbc
import os
import json

import pandas as pd
import urllib.parse
import logging
from datetime import datetime

In [6]:
%run OEA/modules/Ed-Fi/v0.7/src/utilities/edfi_v0_7_edfi_py

In [8]:
%run EdGraph/modules/EdGraph_DW/v0.6/src/utilities/edgraph_dw_v0_6_dw_builder

In [9]:
from datetime import datetime
oea = EdFiOEAChild()   
try:       
    edfi = EdFiClient(workspace, 
                      kvName, 
                      moduleName, 
                      authUrl, 
                      dataManagementUrl, 
                      changeQueriesUrl, 
                      dependenciesUrl, 
                      apiVersion, 
                      batchLimit, 
                      minChangeVer, 
                      maxChangeVer)
except:
    pass

### Global Parameters

In [10]:
def assign_default_variable(variable_name, default_value):
    if variable_name not in globals():
        globals()[variable_name] = default_value
        logger.info(f'{variable_name} not found - using system default')

In [11]:
assign_default_variable(variable_name = 'workspace', 
                        default_value = 'sandbox2')
oea.set_workspace(workspace)

In [12]:
metadata_path = "stage3/EdGraph_DW/build-metadata/stage3-query-assets.json"
metadata_url = oea.to_url(metadata_path)

assign_default_variable(variable_name = 'semantic_schema_name', 
                        default_value = 'semantic')

### Connection Initializations

In [13]:
assign_default_variable(variable_name = 'secret_name', 
                        default_value = 'syn-oea-hisddev-ondemand-mssql-admin-credentials-password')

db_password = oea._get_secret(secret_name)

assign_default_variable(variable_name = 'server_name', 
                        default_value = 'syn-oea-hisddev-ondemand.sql.azuresynapse.net')
assign_default_variable(variable_name = 'database_name', 
                        default_value = 'sdb_sandbox2_s3')

assign_default_variable(variable_name = 'user_name', 
                        default_value = 'eduanalyticsuser')

assign_default_variable(variable_name = 'driver', 
                        default_value = 'ODBC Driver 18 for SQL Server')
password = db_password

db_password = oea._get_secret(secret_name)

### SQL DB Creation - Class Definitions

In [18]:
import pyodbc
import os
import json
import logging
from datetime import datetime

class SQLDatabase:
    def __init__(self, server_name, database_name, user_name, password, driver=driver):
        self.server_name = server_name
        self.database_name = database_name
        self.user_name = user_name
        self.password = password
        self.driver = driver
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        connection_string = f"DRIVER={{{self.driver}}};SERVER={self.server_name};DATABASE={self.database_name};UID={self.user_name};PWD={self.password};"
        return pyodbc.connect(connection_string)

    def execute_query(self, query, isResult = False):
        cursor = self.connection.cursor()
        cursor.execute(query)
        if isResult:
            result = int(cursor.fetchone()[0])
        else:
            result = None
        self.connection.commit()
        cursor.close()
        return result

    def close_connection(self):
        self.connection.close()

class ViewManager:
    def __init__(self, sql_db, data_source):
        self.sql_db = sql_db
        self.data_source = data_source

    def drop_view(self, schema, view_name):
        query = f"DROP VIEW IF EXISTS [{schema}].[{view_name}]"
        self.sql_db.execute_query(query)
    
    def create_schema_if_not_exists(self, schema):
        # NOTE: Under Dev & Review
        query = f"SELECT count(*) FROM sys.schemas WHERE name = N'{schema}';"        
        isSchema = self.sql_db.execute_query(query, isResult = True)
        if isSchema == 0:
            logger.info(f'CREATING SCHEMA {schema}')
            query = f"CREATE SCHEMA [{schema}] AUTHORIZATION [dbo]"
            self.sql_db.execute_query(query)
                
    def create_view(self, schema, view_name, file_path):
        query = f"""CREATE VIEW [{schema}].[{view_name}]
                AS SELECT *
                FROM  
                    OPENROWSET(
                        BULK '{file_path}',
                        DATA_SOURCE = '{self.data_source}',
                        FORMAT='DELTA'
                    ) nyc"""
        self.sql_db.execute_query(query)

    def add_view_to_sql_db(self, schema, view_name, file_path, overwrite=True):
        if overwrite:
            logging.info('Overwrite Mode Active - Dropping SQL DB Table')
            self.drop_view(schema, view_name)
        else:
            logging.info('Overwrite Mode Inactive')
        self.create_view(schema, view_name, file_path)

    def create_and_populate_views(self, schema, schema_path, view_names, overwrite=False):
        self.create_schema_if_not_exists(schema)
        for view_name in view_names:
            logging.info(f'Creating View - {view_name}')
            view_path = f"{schema_path}/{view_name}"
            try:
                self.add_view_to_sql_db(schema, view_name, view_path, overwrite)
            except Exception as error:
                logging.error(f'An error occurred while creating the SQL DB Table - {error}')

### Automated Creations (All Tables)

In [19]:
sql_db = SQLDatabase(server_name, database_name, user_name, db_password)

assign_default_variable(variable_name = 'data_source', 
                        default_value = 'sandbox2_data_source')
                    
root_path = 'stage3/EdGraph_DW'

In [22]:
dbo_path = f"{root_path}/dbo"
auth_path = f"{root_path}/auth"
config_path = f"{root_path}/config"

dbo_tables = oea.get_folders(dbo_path)
auth_tables = oea.get_folders(auth_path)
config_tables = oea.get_folders(config_path)

if workspace == 'prod':
    # FIXME: 2024-03-06 For prod
    root_path = 'EdGraph_DW'
    dbo_path = f"{root_path}/dbo"
    auth_path = f"{root_path}/auth"
    config_path = f"{root_path}/config"

view_manager = ViewManager(sql_db, data_source)

view_manager.create_and_populate_views('dbo', dbo_path, dbo_tables, overwrite=True)
view_manager.create_and_populate_views('auth', auth_path, auth_tables, overwrite=True)
view_manager.create_and_populate_views('config', config_path, config_tables, overwrite=True)

### Semantic Views

In [ ]:
"""jsonDF = spark.read.option("multiline", "true").json(metadata_url).cache()

json_string = jsonDF.toJSON().collect()[0]
original_metadata = json.loads(json_string)"""

In [ ]:
"""semantic_views_builder = SemanticViewsBuilder(original_metadata = original_metadata, 
                                              stage3_db_name = 'PLACEHOLDER',
                                              stage_3_path = 'PLACEHOLDER', 
                                              partitioning = False, 
                                              spark = spark, 
                                              oea = oea, 
                                              logger = logger)
semantic_views_builder.process_metadata()"""

In [ ]:
"""schema_name = 'semantic'
schema_queries_in_order, schema_queries_params_in_order = metadata_processor.return_schema_queries_in_order(schema_name)

table_names = list(schema_queries_in_order.keys())
for table_name in table_names:
    try:
        logger.info(f'TABLE CREATION - {table_name}')
        parameterized_queries = metadata_processor.parameterize_table_queries(
                                                table_name=table_name,
                                                semantic_schema_name = semantic_schema_name)
    
        metadata_processor.execute_table_queries(schema_name,
                                                table_name, 
                                                parameterized_queries,
                                                surrogate_key = True)
        print()
    except Exception as error:
        logger.error(error)
        # logger.error(f"An Error Occurred while creating: {table_name}")"""

### Example (Individual Table)

In [10]:
# sql_db = SQLDatabase(server_name, database_name, user_name, db_password)

# data_source = 'sandbox2_data_source'
# root_path = 'stage3/EdGraph_DW'
# dbo_path = f"{root_path}/dbo"

# view_manager = ViewManager(sql_db, data_source)

In [12]:
# schema = 'dbo'
# view_name = 'DimDate'
# file_path = f"{root_path}/{schema}/{view_name}"
# overwrite = True
# try:
#    view_manager.add_view_to_sql_db(schema = schema, 
#                                    view_name = view_name, 
#                                    file_path = file_path, 
#                                    overwrite = overwrite)
# except Exception as error:
#    logger.error(f'Error - {error}')

In [18]:
# NOTE: This will close connection: Execute only at the end of all your operations
#       In case of accidental execution, reinilize class objects: sql_db, view_manager
# sql_db.close_connection()